In [ ]:
import pandas as pd
from ml_price_model import (
    make_synthetic_training_data,
    train_baseline_model,
    predict_price_drop_probability,
)
from datetime import date


In [ ]:
df = make_synthetic_training_data(n_rows=1000)
df.head()


In [ ]:
df = make_synthetic_training_data(n_rows=1000)
df.head()


In [ ]:
model = train_baseline_model(df)
model

In [ ]:
offer = {
    "origin": "SJU",
    "destination": "JFK",
    "airline": "JetBlue",
    "region": "US",
    "departure_date": "2025-12-10",
    "total_price_usd": 320.0,
}

prob = predict_price_drop_probability(model, offer, search_date=date.today())
prob

In [ ]:
import joblib

joblib.dump(model, "../models/price_drop_model.pkl")